In [1]:
#!pip install -q peft transformers bitsandbytes datasets


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load the model

In [1]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
from transformers import Blip2ForConditionalGeneration, Blip2Processor, BitsAndBytesConfig
from PIL import Image
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

quant_config = BitsAndBytesConfig(load_in_8bit=True)

processor = Blip2Processor.from_pretrained("kkatiz/THAI-BLIP-2")
model = Blip2ForConditionalGeneration.from_pretrained("kkatiz/THAI-BLIP-2", device_map=device, torch_dtype=torch.bfloat16, quantization_config=quant_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00008-of-00009.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00009-of-00009.safetensors:   0%|          | 0.00/604M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/102 [00:00<?, ?B/s]

## Define LoraConfig

In [ ]:
from peft import LoraConfig, get_peft_model

# Let's define the LoraConfig
config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,211,264 || all params: 9,634,960,384 || trainable%: 0.0333


### Dataset Clearing

In [ ]:
# #!cp -r /content/drive/MyDrive/SuperAI-SS5-Hack2/image-processing-thai-language-image-captioning.zip /content/
# !unzip /content/image-processing-thai-language-image-captioning.zip -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/train/train/travel/15347.jpg  
  inflating: /content/train/train/travel/15348.jpg  
  inflating: /content/train/train/travel/15349.jpg  
  inflating: /content/train/train/travel/15350.jpg  
  inflating: /content/train/train/travel/15351.jpg  
  inflating: /content/train/train/travel/15352.jpg  
  inflating: /content/train/train/travel/15353.jpg  
  inflating: /content/train/train/travel/15354.jpg  
  inflating: /content/train/train/travel/15355.jpg  
  inflating: /content/train/train/travel/15356.jpg  
  inflating: /content/train/train/travel/15357.jpg  
  inflating: /content/train/train/travel/15358.jpg  
  inflating: /content/train/train/travel/15359.jpg  
  inflating: /content/train/train/travel/15360.jpg  
  inflating: /content/train/train/travel/15361.jpg  
  inflating: /content/train/train/travel/15362.jpg  
  inflating: /content/train/train/travel/15363.jpg  
  inflating: /content/train/train/travel/15364.j

In [ ]:
# import json
# from datasets import load_dataset

# def read_json_file(filename):
#     try:
#         with open(filename, 'r') as f:
#             return json.load(f)
#     except (FileNotFoundError, json.JSONDecodeError) as e:
#         print(f"Error loading JSON: {e}")
#         return None

# # Load caption data
# data = read_json_file("capgen_v1.0_train.json")

# # Convert JSON keys to just image_id for O(1) lookup
# if data:
#     caption_dict = {k: v for k, v in data.items()}

## Then mapping the folder and the capgen

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        # Access the "caption" column instead of "text"
        encoding["text"] = item["caption"]
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

In [ ]:
from datasets import load_dataset
dataset = load_dataset("saksornr/coco_caption-thai-ipu24-train-sample10k", split="train")

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/398M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/403M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/402M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/399M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
dataset[0]

{'cocoid': '570956',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'caption': 'ภาพจากกระจกข้างรถยนต์ เห็นเป็นผู้ชายสวมเสื้อสีน้ำเงิน สวมกางเกงขายาวกำลังยืนอยู่ข้างรถตู้สีส้ม มีกระติกน้ำสีแดงตั้งอยู่ด้านในรถ'}

In [ ]:
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=3, collate_fn=collate_fn)

In [ ]:
import torch

torch.cuda.empty_cache()



In [ ]:
!nvidia-smi --gpu-reset

GPU 00000000:00:04.0 is currently in use by another process.

1 device is currently being used by one or more other processes (e.g., Fabric Manager, CUDA application, graphics application such as an X server, or a monitoring application such as another instance of nvidia-smi). Please first kill all processes using this device and all compute applications running in the system.


## Setup Train Model

In [ ]:
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()

for epoch in range(200):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device, torch.float16)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

Epoch: 0


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 100.12 MiB is free. Process 3271 has 14.64 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 113.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Inference
Let's check the results on our train dataset

In [ ]:
# load image
example = dataset[0]
image = example["image"]
image

Image Captioning

In [ ]:
# prepare image for the model
img = Image.open()
inputs = processor(images=img, return_tensors="pt").to(device, torch.float16)
pixel_values = inputs.pixel_values

# Adjust your `max_length`
generated_ids = model.generate(**inputs, max_length=25)
generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
# generated_ids = model.generate(**inputs, max_length=20)
# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_caption)